In [1]:
import pandas as pd
import json
from utils import *

# Load Files

In [2]:
# Read file
data_csv  = pd.read_csv('./../pct_wn_first_lot.csv')
read_file = open("./../ner_lot1_110520_num_word_192449_num_tag_20702.json", "r")
data_json = json.load(read_file)

# Tokenize entities and words

In [3]:
from attacut import tokenize, Tokenizer

In [4]:
def tokenize_entities2word(set_index, txt_test_data):
    # Dumps and load json data
    txt_test_data      = json.dumps(txt_test_data)
    txt_test_data      = json.loads(txt_test_data)
    temp               = 0
    save_token         = []
    
    for idx, _ in enumerate(range(len(txt_test_data))):
        
        # Tokenize entities
        if idx in set_index:
            e_token = txt_test_data[temp:idx]

            #Check empty text
            if(len(e_token) > 0):
                
                # Tokenize each sentence
                words_tokenize = tokenize(e_token)
                save_token.extend(words_tokenize)
                temp = idx
    save_token.extend(tokenize(txt_test_data[temp:]))
    return save_token

# Tags top entities level by BIOS scheme 

In [5]:
def get_entity_index(idx, entities_list):
    tags  = []
    for idx_ne, entity in enumerate(entities_list):
        en_type, en_idx = entity
        
        # Return first entity
        if(idx in range(en_idx[0],en_idx[1])):
            return (idx_ne,'', en_type)
        
    return (-1,'O', '')

In [6]:
def save_BIOS_tokenzie(word, tag, file):
    save_format   = '{} {}\n'.format(word, tag[1]+tag[2])
    file.write(str(save_format))
    
def tags_BIOS_top_entities_level(word_token, en_boundary, file):
    count_top_en  = 0
    words_tags    = []
    shift_index   = 0
    BIOS_tags     = ''
    
    # Initial value NE1 NE2
    NE1           = get_entity_index(-1, en_boundary)
    NE2           = get_entity_index(shift_index, en_boundary)
    
    if((NE1[1] in ['S', 'E'] or NE1[0] == -1) and NE2[0] != -1): BIOS_tags = 'S'
    NE2           = (NE2[0], BIOS_tags, NE2[2])


    for word in word_token:
        BIOS_tags   = ''
        shift_index += len(word)
        
        # Get tag NE3
        NE3         = get_entity_index(shift_index, en_boundary)
        NE3         = (NE3[0], BIOS_tags, NE3[2])

        # Initial value
        if( NE3[0] != -1 ):
            #'SEO->SOB'
            if(NE2[1] in ['S', 'E'] or NE2[0] == -1):
                BIOS_tags = 'S'

            #'BI->IE'
            elif(NE2[1] in ['B', 'I']): 
                BIOS_tags = 'I'
        else:
            BIOS_tags = 'O'

        NE3 = (NE3[0], BIOS_tags, NE3[2])

        # Update NE2 and NE3
        if(NE2[0] == NE3[0]):
            if(NE2[1] == 'S'):
                NE2   = (NE2[0], 'B', NE2[2])
                NE3   = (NE3[0], 'I', NE3[2])

        elif(NE2[1] == 'I'):
                NE2   = (NE2[0], 'E', NE2[2])

        # Shift NE2 to NE1
        if(NE2[0] != -1):
            NE1 = (NE2[0], NE2[1]+'-', NE2[2])
        else :
            NE1 = (NE2[0], NE2[1], NE2[2])

        # Shift NE3 to NE2
        NE2 = NE3
        
        # Save BIOS tokenize
        save_BIOS_tokenzie(word, NE1, file)
        
        # Count top-level tokens
        if(NE1[0] != -1):
            if(NE1[1] in ['S-', 'E-']): 
                count_top_en+=1
        
        
    # Add stop sentence and close file
    file.write('\n')
    
    return count_top_en

# Tags and save data

In [7]:
# Load data 
tags_ne_json      = data_json
segment_text      = data_csv['ssg']
clean_text        = data_csv['text_clean']

# Remove pipe
clean_data        = remove_pipe(tags_ne_json, segment_text, clean_text)

# Open file for save data
file = open('example_data_word_BIOS_Tagging.txt', 'w')

# Count word and sentence
word_count   = 0
sent_count   = 0
top_en_count = 0

for idx, data in enumerate(clean_data):
    
    # Prepare data
    entities            = data['entities']
    text                = data['text']
    set_index_boundary  = get_all_entities_index(entities)
    en_boundary         = get_entities_boundary(entities)

    # Tokenize word
    words_token         = tokenize_entities2word(set_index_boundary, text)
    # Count word
    word_count          += len(words_token)
    sent_count          += 1
    top_en_count        += tags_BIOS_top_entities_level(words_token, en_boundary, file)
    
# Close file
file.close
    
print('\n\nResults : {} sentences, {} words, {} top-level tokens'.format(sent_count, word_count, top_en_count))



Results : 800 sentences, 209326 words, 10860 top-level tokens


# Check entities from raw data

In [8]:
sentence_no  = 0
tags_ne_json = sorted(data_json[sentence_no]['entities'], key=lambda x:(x['start_idx'], -x['end_idx']))
print(data_json[sentence_no]['text'],'\n')
[print('{:>5} : {:<5} {:20} {}'.format(x['start_idx'], x['end_idx'], x['type'], x['text'])) for x in tags_ne_json]
pass

Sukishi  สา ขา Mega Bangna  ตัว ร้าน จะ อยู่ โซน  food walk  ค่ะ จะ อยู่ ใกล้ ๆ กับ  star buck  เลย  ครั้ง นี้ มา ตรง กับ วัน หยุด  อืัอ หือ ออ คน อย่าง แน่น เลย ย  ต้อง รอ คิว ค่ะ  วัน นี้ รอ3 คิว  ก็ รอ ไม่ นาน แปป เดียว  ตัว ร้าน ที่ สา ขา นี้ ถือ ว่า กว้าง มาก ๆ เลย ค่ะ  จัด ร้าน ดู โปร่ง โล่ง สบาย ด้วย  ช่วง นี้ มี เฮ    เพราะ เค้า ลด รา คา สำ หรับ อา หาร หลาย ๆ ชุด ค่ะ  ก็ จัด มา1 ชุด  จำ รา คา ไม่ ได้ ไม่ แน่ ใจ ว่า  799.  หรือ เปล่า  อ้ะ ะะ หิว มาก ก็ จัด ซะ เลย ย  เนื้อ นุ่ม มาก อร่อย ด้วย คือ ดี คือ ฟิน นน น  ยิ่ง เป็น เบ คอน ยิ่ง อร่อย เลย ค่ะ  รส ชาติ หมู และ ความ นุ่ม ยก ให้ สุ กี้ ชิ เลย  เครื่อง ดื่ม  สั่ง ชา มะ นาว และ โก โก้ เฟรป เป้ มา  โก โก้ รส ชาติ ใช้ ได้ เลย ค่ะ  แต่ หวาน มาก ไป นิด นึง  นึก ถึง ปิ้ง ย่าง นึก ถึง  Sukishi  เลย ค่ะ ยก ใจ ให้ เลย 

    0 : 7     restaurant           Sukishi
   15 : 26    facility:other       Mega Bangna
   15 : 19    loc:others           Mega
   20 : 26    district             Bangna
   50 : 59    loc:others           food walk
   

# Show example data after remove pipe

In [9]:
# Show example data after remove pipe
sentence_no  = 0
check_sentence(clean_data[sentence_no])

Text:
	Sukishi สาขา Mega Bangna ตัวร้านจะอยู่โซน food walk ค่ะจะอยู่ใกล้ๆกับ star buck เลย ครั้งนี้มาตรงกับวันหยุด อืัอหือออคนอย่างแน่นเลยย ต้องรอคิวค่ะ วันนี้รอ3คิว ก็รอไม่นานแปปเดียว ตัวร้านที่สาขานี้ถือว่ากว้างมากๆเลยค่ะ จัดร้านดูโปร่งโล่งสบายด้วย ช่วงนี้มีเฮ  เพราะเค้าลดราคาสำหรับอาหารหลายๆชุดค่ะ ก็จัดมา1ชุด จำราคาไม่ได้ไม่แน่ใจว่า 799. หรือเปล่า อ้ะะะหิวมากก็จัดซะเลยย เนื้อนุ่มมากอร่อยด้วยคือดีคือฟินนนน ยิ่งเป็นเบคอนยิ่งอร่อยเลยค่ะ รสชาติหมูและความนุ่มยกให้สุกี้ชิเลย เครื่องดื่ม สั่งชามะนาวและโกโก้เฟรปเป้มา โกโก้รสชาติใช้ได้เลยค่ะ แต่หวานมากไปนิดนึง นึกถึงปิ้งย่างนึกถึง Sukishi เลยค่ะยกใจให้เลย


text                          		type                 start_idx <-> end_idx
            
Sukishi                        		restaurant               0 <-> 7                    --Sukishi--
Mega Bangna                    		facility:other          13 <-> 24                   --Mega Bangna--
Mega                           		loc:others              13 <-> 17                   --Mega--
Bangna     

# Example data

In [10]:
! head example_data_word_BIOS_Tagging.txt

Sukishi S-restaurant
  O
สาขา O
  O
Mega B-facility:other
  I-facility:other
Bangna E-facility:other
  O
ตัว O
ร้าน O
